In [2]:
#install upgraded google-cloud-storage
#pip install --upgrade google-cloud-storage

Requirement already up-to-date: google-cloud-storage in /opt/conda/lib/python3.7/site-packages (1.32.0)
Note: you may need to restart the kernel to use updated packages.


# Access Google Cloud Storage (GCS) data

In [14]:
#Access data files for H23 and parental test samples (first batch)

from google.cloud import storage
#google apis documentation: https://googleapis.dev/python/storage/latest/index.html
bucket_name = 'sbi-sheffels-e'
#prefix = 'runs/20210113-results/'

storage_client = storage.Client() 
bucket = storage_client.get_bucket(bucket_name)
#Get list of files in data/pE2.N54.fastq.00 folder of bucket
#files = list(storage_client.list_blobs(bucket, prefix=prefix)) #blob = GCS name for file/object
#for file in files:
 #   print(file) #print "blob" names



In [17]:
stringtie_file = "runs/20210113-results/stringtieFPKM/ballgown/E2-78710_S1_L004_R1_001Aligned.sortedByCoord.out_ballgown/t_data.ctab"
stringtie_filename = "t_data.ctab"
featureCount_file = "runs/20210113-results/featureCounts/gene_counts/E2-78710_S1_L004_R1_001Aligned.sortedByCoord.out_gene.featureCounts.txt"
featureCount_filename = "78710_featureCount.txt"

In [18]:
#get files
blob_stringtie = storage.blob.Blob(stringtie_file,bucket)
#download blob to AI Platform Notebook instance
with open(stringtie_filename, "wb") as file_obj:
    blob_stringtie.download_to_file(file_obj)

blob_featureCount = storage.blob.Blob(featureCount_file,bucket)
#download blob to AI Platform Notebook instance
with open(featureCount_filename, "wb") as file_obj:
    blob_featureCount.download_to_file(file_obj)


In [27]:
import pandas as pd
from statistics import mean

In [3]:
featureCount = pd.read_csv("78710_featureCount.txt", sep="\t", header = 1)

In [4]:
featureCount.columns

Index(['Geneid', 'Chr', 'Start', 'End', 'Strand', 'Length', 'gene_name',
       'E2-78710_S1_L004_R1_001Aligned.sortedByCoord.out.bam'],
      dtype='object')

In [5]:
featureCount["Counts"]=featureCount["E2-78710_S1_L004_R1_001Aligned.sortedByCoord.out.bam"]

In [6]:
featureCount.drop(axis=1, columns=['Chr', 'Start', 'End', 'Strand', 'Length',
       'E2-78710_S1_L004_R1_001Aligned.sortedByCoord.out.bam'], inplace=True)

In [7]:
featureCount

,Geneid,gene_name,Counts
0,ENSG00000223972,DDX11L1,0
1,ENSG00000227232,WASH7P,276
2,ENSG00000243485,MIR1302-10,0
3,ENSG00000237613,FAM138A,0
4,ENSG00000268020,OR4G4P,0
...,...,...,...
63672,ENSG00000224240,CYCSP49,0
63673,ENSG00000227629,SLC25A15P1,0
63674,ENSG00000237917,PARP4P1,0
63675,ENSG00000231514,FAM58CP,0


In [8]:
featureCount.sort_values("gene_name", inplace=True)

In [9]:
featureCount.loc[featureCount["gene_name"]=="AACS"]

,Geneid,gene_name,Counts
13481,ENSG00000081760,AACS,2216


In [10]:
featureCount["normalized_counts"]=featureCount["Counts"]/2216

In [11]:
stringtieCount = pd.read_csv("stringtie_counts.csv")

In [12]:
stringtieCount.rename({"Unnamed: 0": "Gene_name"}, axis =1, inplace=True)

In [13]:
compare = pd.DataFrame()
compare["Gene"]=stringtieCount["Gene_name"]

In [14]:
stringtieCount

,Gene_name,abundance,counts,length,countsFromAbundance,counts_normalized
0,5S_rRNA,0.000000,0.000000,197.741935,no,0.000000
1,7SK,0.184337,60.100913,2775.000000,no,0.011352
2,A1BG,3.835476,226.055937,501.639247,no,0.042698
3,A1BG-AS1,1.681708,235.714062,1192.970295,no,0.044522
4,A1CF,0.000000,0.000000,2450.400000,no,0.000000
...,...,...,...,...,...,...
56631,ZYG11AP1,0.000000,0.000000,837.000000,no,0.000000
56632,ZYG11B,6.432696,6022.389568,7968.389620,no,1.137518
56633,ZYX,130.485857,32288.723414,2106.115553,no,6.098740
56634,ZZEF1,5.771518,7329.780256,10809.252789,no,1.384459


In [15]:
compare["Stringtie"]=stringtieCount["counts_normalized"]

In [49]:
compare["featureCount"]=featureCount.loc[featureCount["gene_name"]==compare["Gene"]]

ValueError: Can only compare identically-labeled Series objects

In [31]:
#gene_list = compare["Gene"].tolist()
non_zero_list = []
value_list = []
def make_avg_value(row):
    gene = row["Gene"]
    value_list = []
    non_zero_list = []
    value_list = featureCount.loc[featureCount["gene_name"]==gene]["normalized_counts"].tolist()
    for value in value_list:
        if value > 0:
            non_zero_list.append(value)
    if len(non_zero_list) == 0:
        gene_average = 0
    elif len(non_zero_list) == 1:
        gene_average = non_zero_list[0]
    elif len(non_zero_list) > 1:
        gene_average = mean(non_zero_list)
    return gene_average

In [41]:
def make_avg_value(row):
    gene = row["Gene"]
    value_list = featureCount.loc[featureCount["gene_name"]==gene]["normalized_counts"].tolist()
    gene_average = mean(value_list)
    return gene_average

In [42]:
compare["featureCount_with0"] = compare.apply(lambda row: make_avg_value(row), axis=1)

In [49]:
compare.loc[abs(compare["featureCount"]-compare["Stringtie"])>(.1*compare["Stringtie"])].loc[compare["Stringtie"]>20]

,Gene,Stringtie,featureCount,featureCount_with0
14799,EEF1A1,481.814875,401.810469,401.810469
14833,EEF1A1P5,61.478640,0.808213,0.808213
14852,EEF1G,45.863584,0.000000,0.000000
15024,EIF4B,22.274156,19.733755,19.733755
16782,GAPDH,346.996023,250.412455,250.412455
19979,ITGB1,22.378251,18.745036,18.745036
24934,MT-ATP8,86.415585,4.680054,4.680054
24935,MT-CO1,50.031114,42.914260,42.914260
24943,MT-ND4L,2535.957351,13.684116,13.684116
24949,MT-TC,75.402070,0.759928,0.759928


In [50]:
compare

,Gene,Stringtie,featureCount,featureCount_with0
0,5S_rRNA,0.000000,0.000000,0.000000
1,7SK,0.011352,0.015343,0.001279
2,A1BG,0.042698,0.003610,0.003610
3,A1BG-AS1,0.044522,0.028881,0.028881
4,A1CF,0.000000,0.000000,0.000000
...,...,...,...,...
56631,ZYG11AP1,0.000000,0.000000,0.000000
56632,ZYG11B,1.137518,1.138087,1.138087
56633,ZYX,6.098740,5.825361,5.825361
56634,ZZEF1,1.384459,1.391245,1.391245


In [51]:
#conclusion: featureCount and Stringtie are different, but will proceed with featureCount
#significant (>10%) differences in 22 genes out of 56636